In [1]:
import os

if "notebooks" in os.getcwd():
    os.chdir(os.path.abspath(os.path.join(os.getcwd(), "..")))

import ollama
from IPython.display import Markdown, display, clear_output
import re
import tqdm
import lib.utils
import datetime

dict_d = {
    'ciencias-humanas' : 'H',
    'matematica' : 'M',
    'ciencias-natureza' : 'N',
    'linguagens' : 'L'
}

questoes = lib.utils.load_json('./data/questoes/questoes.json')

new_questions = lib.utils.load_json('./data/questoes.json')

primary_model = 'gemma2'
secundary_model = 'minicpm-v'


In [2]:
import re

def extrair_habilidades_e_assuntos(mensagem):
    # Define o padrão regex para capturar habilidades e assuntos
    padrao = r'<hab>(.*?)</hab>|<as>(.*?)</as>'
    
    # Usa finditer para encontrar todas as correspondências no texto
    matches = re.finditer(padrao, mensagem)
    
    habilidades = []
    assuntos = []
    
    for match in matches:
        if match.group(1):  # Captura o conteúdo da tag <hab>
            habilidades.append(match.group(1))
        elif match.group(2):  # Captura o conteúdo da tag <as>
            assuntos.append(match.group(2))
    
    return habilidades, assuntos

In [3]:
from lib.models_help.build import get_messages, context_description_image, answer_description_image, get_images
from lib.models_help.habilities import dict_assuntos, dict_habilidades

def get_habilidades_e_assuntos(question):

    descriptions = []

    images = get_images(question)

    if question['type'] in ['context-image', 'full-image']:
        print("Contexto da questão enviados as: ", datetime.datetime.now())
        response = ollama.generate(
            model=secundary_model,
            prompt= context_description_image(question),
            images= [images.pop(0)]
        )
        descriptions.append(response.response)

    if question['type'] in ['answer-image', 'full-image']:
        for i, ans in enumerate(["A", "B", "C", "D", "E"], start=1):
            print(f"Alternativa {ans} da questão enviados as: ", datetime.datetime.now())
            ans_prompt_str = answer_description_image(question, ans)
            ans_response = ollama.generate(
                model=secundary_model, 
                prompt=ans_prompt_str,
                images= [images.pop(0)]
            )
            descriptions.append(ans_response.response)

    message = [{"role" : "system", "content" : f"Você é um grande especialista em {question['discipline']} da prova do ENEM. Dada a questão abaixo, responda"}]

    # Contexto da questão
    if question['type'] in ('full-image', 'context-image'):
        if images:
            message.append(
                {"role" : "user", "content" : "Esta é a imagem da questão: ", "images" : [images.pop(0)]}
            )
        elif descriptions:
            message.append(
                {"role" : "user", "content" : "Esta é a descrição da questão: ", "content" : descriptions.pop(0)}
            )

    # Introdução à alternativa
    message.append(
        {"role" : "system", "content" : f"Observe com atenção a introdução das alternativas da questão: {question['alternatives_introduction']}"}
    )

    # Alternativas
    for alternative in ["A", "B", "C", "D", "E"]:
        if images:
            message.append(
                {"role" : "user", "content" : f"({alternative}) : {question[alternative]} - Imagem: ", "images" : [images.pop(0)]}
            )
        elif descriptions:
            message.append(
                {"role" : "user", "content" : f"({alternative}) : {question[alternative]} - Descrição da Imagem: {descriptions.pop(0)}"}
            )
        else:
            message.append(
                {"role" : "user", "content" : f"({alternative}) : {question[alternative]}"}
            )

    discipline = question['discipline']
    habilidades = dict_habilidades.get(discipline, '')

    message.append({
        "role" : "system", 
        "content" : f"Observe a lista de habilidades referentes à disciplina de {discipline}"
    })

    message.append({
        "role" : "user", 
        "content" : str(habilidades)
    })

    message.append({
        "role" : "system", 
        "content" : f"Observe a lista de assuntos referentes à disciplina de {discipline}"
    })

    message.append({
        "role" : "user", 
        "content" : str(habilidades)
    })

    assuntos = dict_assuntos.get(discipline, '')
    message.append({
        "role" : "user", 
        "content" : str(assuntos)
    })

    message.append({
        "role" : "system", 
        "content" : """
    Você deve identificar até 3 habilidades principais e 3 assuntos relacionado ao texto. Sua resposta deve ser no seguinte formato de tags:

    Essas habilidades e assuntos devem estar no 

    <hab>HX</hab>
    Exemplo: <hab>H1</hab>, <hab>H2</hab>, <hab>H3</hab>

    <as>Assunto Y</as>
    Exemplo:
    <as>Origem e evolução da vida</as>, <as>Ecologia e ciências ambientais</as>, <as>Identidade dos seres vivos - Níveis de organização dos seres vivos</as>

    Responda com a habilidade e o assunto mais relevantes para o texto acima. Se o assunnto/Habilidade já estiver na lista abaixo, ponha da mesma forma como está na lista, mas dentro da TAG.
    """
    })

    response = ollama.chat(
        model=primary_model,
        messages=message,
        stream=False,
    )
    
    return extrair_habilidades_e_assuntos(response.message.content)

In [4]:
from lib.models_help.build import get_messages, context_description_image, answer_description_image, get_images

for question in tqdm.tqdm((list(q for q in questoes if str(q['id']) not in new_questions.keys()))):
    
    if str(question['id']) in new_questions.keys():
        continue
    
    question['skills'], question['subjects'] = get_habilidades_e_assuntos(question)
    
    new_questions[str(question['id'])] = question
    
    lib.utils.update_json(new_questions, './data/questoes.json')
    clear_output(wait=True)

100%|██████████| 168/168 [20:50<00:00,  7.44s/it]
